In [1]:
from faker import Faker
from PIL import Image, ImageFont, ImageDraw
import random
import augraphy as ag
import numpy as np
import os
from tqdm import trange

In [2]:
gen = Faker('th')

In [3]:
gen.random_number()

897998725

In [4]:
method_names = (
    'address','aba','administrative_unit',
    'amphoe', 'city','color_name',
    'company', 'date', 'day_of_week', 'first_name',
    'first_name_female', 'first_name_male', 'last_name',
    'license_plate', 'street_address', 'street_name',
    'suffix', 'tambon', 'time', 'random_number', 'prefix'
)
methods  = [gen.__getattr__(m) for m in method_names]
methods += [lambda: gen.sentence(5),
            lambda: gen.prefix() + gen.random_element(['', ' ']) + gen.name(),
            lambda: gen.prefix() + gen.random_element(['', ' ']) + gen.first_name(),
           ]

In [5]:
def arabic2th(n):
    return chr(ord(n)+(ord('๑')-ord('1')))

In [6]:
font = ImageFont.truetype('fonts/THSarabun Bold.ttf', size=50)

In [7]:
fonts = ['fonts/'+f for f in os.listdir('fonts/') if f.endswith('.ttf')]
def get_random_font():
    return ImageFont.truetype(random.choice(fonts), size=random.randint(20,100))

In [8]:
def get_random_text():
    text = gen.random_element(methods)()
    if not isinstance(text, str):
        text = str(text)
    if ''.isalnum() and random.random() < .5:
        text = ''.join([arabic2th(c) if c.isdigit() else c for c in text])
    return text
    
def gen_image_text():
    font = get_random_font()
    text = get_random_text()
    x0,y0,x1,y1 = font.getbbox(text,)
    px0,py0,px1,py1 = [random.randint(0, 20) for _ in range(4)] # padding
    image_c, text_c = random.choice([['white', 'black'],])
    image = Image.new('RGB', (x1+px0+px1, y1+py0+py1), image_c)
    draw = ImageDraw.Draw(image)
    draw.text((px0,py0), text, fill=text_c, font=font,)
    return image, text

In [9]:
aug_pipeline = ag.AugmentationSequence([
    ag.LowInkRandomLines(),
    ag.BleedThrough(),
    ag.DirtyDrum(),
    ag.Gamma(gamma_range=(.1, .3)),
    ag.LowInkPeriodicLines((1,1), (3,10)),
    ag.DirtyDrum(),
    ag.Dithering(p=.3),
])

In [10]:
def aug(image: Image):
    return Image.fromarray(aug_pipeline(np.array(image))[0])

In [14]:
def gen_and_aug():
    img, txt = gen_image_text()
    return [aug(img) for _ in range(3)], txt

In [15]:
output_dir = 'output/textrecog/imgs'
os.makedirs(output_dir, exist_ok=True)
size = 400_000
img_id = 0
fp = open(os.path.join(output_dir, 'metadata.txt'), 'w')
fp.write('file_name,text\n')
for i in trange(size):
    imgs, text = gen_and_aug()
    for img in imgs:
        img_name = f'{img_id}.png'
        img_id += 1
        img.save(os.path.join(output_dir,img_name), quality=100, subsampling=0)
        fp.write(f'{img_name},{text}\n')
fp.close()   

  3%|█                                 | 12259/400000 [17:52<9:25:26, 11.43it/s]


KeyboardInterrupt: 